In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 2.7.15 (default, Oct  2 2018 11:42:04)
SparkSession available as 'spark'.


## Extract the column names

In [20]:
datafile = "green_tripdata_2018-06.csv"
with open(datafile) as f:
    first_line = f.readline()
col_names

[u'VendorID',
 u'lpep_pickup_datetime',
 u'lpep_dropoff_datetime',
 u'store_and_fwd_flag',
 u'RatecodeID',
 u'PULocationID',
 u'DOLocationID',
 u'passenger_count',
 u'trip_distance',
 u'fare_amount',
 u'extra',
 u'mta_tax',
 u'tip_amount',
 u'tolls_amount',
 u'ehail_fee',
 u'improvement_surcharge',
 u'total_amount',
 u'payment_type',
 u'trip_type']

## Filter out the useful columns from dataset

In [21]:
column_map = {}
useful_columns = ["pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "pickup_datetime","pickup_latitude", "pickup_longitude","dropoff_datetime","dolocationid", "pulocationid", "fare_amount", "trip_distance", "passenger_count"]

for i, c in enumerate(col_names):
    c = c.lower()
    for u in useful_columns:
        if c in u or u in c:
            column_map[u] = i
column_map

{'dolocationid': 6,
 'dropoff_datetime': 2,
 'fare_amount': 9,
 'passenger_count': 7,
 'pickup_datetime': 1,
 'pulocationid': 5,
 'trip_distance': 8}

## Create a dataframes from rdds. Cast them to appropriate type. Drop null valued rows

In [126]:
from pyspark.sql import Row
from pyspark.sql.types import FloatType, IntegerType
from pyspark.sql.functions import col

rdd1 = sc.textFile(datafile).map(lambda line: line.split(",")).filter(lambda line: len(line)>1)

if 'dolocationid' in column_map:
    df = rdd1.map(lambda line: Row(pulocationid=line[column_map['pulocationid']], 
                              dolocationid=line[column_map['dolocationid']], 
                              pickup_datetime=line[column_map['pickup_datetime']], 
                              dropoff_datetime=line[column_map['dropoff_datetime']],
                              trip_distance=line[column_map['trip_distance']], 
                              fare_amount=line[column_map['fare_amount']], 
                              passenger_count=line[column_map['passenger_count']])).toDF()
    df = df.withColumn("dolocationid", df["dolocationid"].cast(IntegerType()))
    df = df.withColumn("fare_amount",df["fare_amount"].cast(FloatType()))
    df = df.withColumn("passenger_count", df["passenger_count"].cast(IntegerType()))
    df = df.withColumn("pulocationid", df["pulocationid"].cast(IntegerType()))
    df = df.withColumn("trip_distance", df["trip_distance"].cast(FloatType()))
    
    df = df.filter(col("dolocationid").isNotNull() & col("dropoff_datetime").isNotNull() & \
                   col("fare_amount").isNotNull() & col("passenger_count").isNotNull() & \
                   col("pulocationid").isNotNull() & col("trip_distance").isNotNull() & col("pickup_datetime").isNotNull())

else:
    df = rdd1.map(lambda line: Row(pickup_longitude=line[column_map['pickup_longitude']], 
                                  pickup_latitude=line[column_map['pickup_latitude']], 
                                  dropoff_longitude=line[column_map['dropoff_longitude']], 
                                  dropoff_latitude=line[column_map['dropoff_latitude']], 
                                  pickup_datetime=line[column_map['pickup_datetime']], 
                                  dropoff_datetime=line[column_map['dropoff_datetime']], 
                                  trip_distance=line[column_map['trip_distance']], 
                                  fare_amount=line[column_map['fare_amount']], 
                                  passenger_count=line[column_map['passenger_count']])).toDF()
    df = df.withColumn("dropoff_longitude", df["dropoff_longitude"].cast(FloatType()))
    df = df.withColumn("dropoff_latitude", df["dropoff_latitude"].cast(FloatType()))
    df = df.withColumn("fare_amount",df["fare_amount"].cast(FloatType()))
    df = df.withColumn("passenger_count", df["passenger_count"].cast(IntegerType()))
    df = df.withColumn("pickup_longitude", df["pickup_longitude"].cast(FloatType()))
    df = df.withColumn("pickup_latitude", df["pickup_latitude"].cast(FloatType()))
    df = df.withColumn("trip_distance", df["trip_distance"].cast(FloatType()))
    df = df.filter(col("dropoff_longitude").isNotNull() & col("dropoff_datetime").isNotNull() & \
                   col("fare_amount").isNotNull() & col("passenger_count").isNotNull() & \
                   col("dropoff_longitude").isNotNull() & col("trip_distance").isNotNull() & \
                   col("pickup_latitude").isNotNull() & col("pickup_longitude").isNotNull() & \
                   col("pickup_datetime").isNotNull())

df.show()


+------------+-------------------+-----------+---------------+-------------------+------------+-------------+
|dolocationid|   dropoff_datetime|fare_amount|passenger_count|    pickup_datetime|pulocationid|trip_distance|
+------------+-------------------+-----------+---------------+-------------------+------------+-------------+
|          33|2018-06-01 00:36:13|        4.0|              5|2018-06-01 00:33:55|          66|         0.51|
|          49|2018-06-01 00:49:46|        9.0|              5|2018-06-01 00:40:36|          25|         1.97|
|          49|2018-06-01 01:02:58|        6.5|              5|2018-06-01 00:57:12|          61|          1.4|
|          97|2018-06-01 00:16:27|        7.0|              1|2018-06-01 00:10:13|          49|         1.36|
|         127|2018-06-01 00:52:06|       24.0|              1|2018-06-01 00:32:08|          75|          7.9|
|         112|2018-06-01 00:27:44|       11.0|              2|2018-06-01 00:15:35|          36|          2.9|
|         

## Get the pickup and dropoff zipcode for the trip

In [127]:
if 'dolocationid' in column_map:
    import csv
    from itertools import chain
    from pyspark.sql.functions import create_map, lit
    
    mapFile = 'locationid_zipcode_map.csv'
    reader = csv.reader(open(mapFile, 'r'))
    zipcodeMap = {}
    reader.next()
    for row in reader:
        try:
            zipcodeMap[int(row[0])] = int(row[1])
        except ValueError:
            continue

    mapping_expr = create_map([lit(x) for x in chain(*zipcodeMap.items())])
    df = df.withColumn("dropoff_zipcode", mapping_expr[df["dolocationid"]])
    df = df.withColumn("pickup_zipcode", mapping_expr[df["pulocationid"]])
    df = df.drop("dolocationid", "pulocationid")
#     df['dropoff_zipcode'] = df.apply(lambda x: zipcodeMap(x['dolocationid']), axis=1)
#     df['pickup_zipcode'] = df.apply(lambda x: zipcodeMap(x['pulocationid']), axis=1)
else:
    from uszipcode import SearchEngine
    from uszipcode import Zipcode
    def get_zip_code(lat,lng):
        result = search.by_coordinates(lat, lng, radius=5, returns=1)
        return result[0]
                
    df['dropoff_zipcode'] = df.apply(lambda x: zipcodeMap(x['dropoff_latitude'], x['dropoff_longitude']), axis=1)
    df['pickup_zipcode'] = df.apply(lambda x: zipcodeMap(x['pickup_latitude'], x['pickup_longitude']), axis=1)

In [128]:
df.show()

+-------------------+-----------+---------------+-------------------+-------------+---------------+--------------+
|   dropoff_datetime|fare_amount|passenger_count|    pickup_datetime|trip_distance|dropoff_zipcode|pickup_zipcode|
+-------------------+-----------+---------------+-------------------+-------------+---------------+--------------+
|2018-06-01 00:36:13|        4.0|              5|2018-06-01 00:33:55|         0.51|          11201|         11201|
|2018-06-01 00:49:46|        9.0|              5|2018-06-01 00:40:36|         1.97|          11205|         11201|
|2018-06-01 01:02:58|        6.5|              5|2018-06-01 00:57:12|          1.4|          11205|         11238|
|2018-06-01 00:16:27|        7.0|              1|2018-06-01 00:10:13|         1.36|          11205|         11205|
|2018-06-01 00:52:06|       24.0|              1|2018-06-01 00:32:08|          7.9|          10034|         10029|
|2018-06-01 00:27:44|       11.0|              2|2018-06-01 00:15:35|          2